In [2]:
import tensorflow as tf               # TesorFlow
import numpy as np                    # NumPy
import matplotlib.pyplot as plt       # Графики-то строить будем
import pandas as pd                   # Для работы с таблицами и CSV-файлами
import sklearn.metrics                # Для построения confusion matrix
import seaborn as sns                 # Для отображения confusion matrix
import os                             # Для возможности выбора GPU

# Библиотека для упрощения работы с SVD
# В колабе ее можно поставить командой !pip install git+https://github.com/mayukh18/reco.git
import reco.recommender

In [3]:
# Используем GPU 0
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Отключаем отладочные сообщения в TensorFlow
tf.get_logger().setLevel('ERROR')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = "2"

In [4]:
DATASET_DIR = '/opt/datasets/movielens'

In [5]:
df = pd.read_csv(DATASET_DIR + '/ratings.csv')
df

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [22]:
utility_matrix = df.pivot(index="userId", columns="movieId", values='rating') #.fillna(0)
utility_matrix

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,2.5,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
u, s, v = np.linalg.svd(utility_matrix.values, full_matrices=False)

Да, можно так...

Но, можно сделать проще - использовать библиотеку [reco](https://github.com/mayukh18/reco)

Создаем объект SDV-рекомендатор, указывая количество фич в разложении

In [23]:
svd = reco.recommender.SVDRecommender(no_of_features=8)

Создаем из DataFrame *Utility Matrix*, указывая в formatizer какой столбец содержит пользователей, какой item-ы, а какой рейтинги

In [24]:
user_item_matrix, users, items = svd.create_utility_matrix(df, formatizer={'user':'userId', 'item':'movieId', 'value':'rating'})

Вот такая *Utility Matrix* получается:

In [25]:
user_item_matrix

array([[4. , nan, 4. , ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [2.5, 2. , 2. , ..., nan, nan, nan],
       [3. , nan, nan, ..., nan, nan, nan],
       [5. , nan, nan, ..., nan, 4. , nan]])

Делаем SVD-разложение *Utility Matrix* на u, s, v. Так же необходимо передать массивы users и items.
И необязательно передавать данные, полученные на предыдущем шаге, можно сделать их самому (если условия задачи более сложные)

In [26]:
svd.fit(user_item_matrix, users, items)

или используем для разложения уже созданную в pandas utility_matrix:

In [28]:
svd.fit(utility_matrix.to_numpy(), utility_matrix.index.values.tolist(), list(utility_matrix.columns))

Теперь нам доступны все возможности SVD-разложения!

Например, можно получить 5 пользователей похожих по item-ам (оценкам/покапкам/ и т.п.) на пользовтеля с userID=1 (отобразятся id похожих пользавтелей и их векторная близость)

In [29]:
svd.topN_similar(x=1, N=5, column='user')

[(555, 0.4449888226854942),
 (201, 0.6172590367318571),
 (453, 0.6305270109229381),
 (186, 0.7169010737448965),
 (93, 0.7285443728759976)]

Или можно получить 7-ми item-ов, похожих на itemID=3 по их использованию/оценкам пользователями:

In [30]:
svd.topN_similar(x=3, N=7, column='item')

[(2012, 0.27226832882353874),
 (1028, 0.3011053980250963),
 (5103, 0.34012564092614406),
 (785, 0.3741251663504187),
 (2916, 0.3958923146917345),
 (2804, 0.4001426062572117),
 (1259, 0.40057537507935215)]

Ну, и конечно же можно получить рекомендации (например 4 рекомендации) по item-ам, для пользователя UserId = 5:

In [31]:
svd.recommend([5], N=4)

[[2972, 6983, 126088, 124851]]

Можно даже сразу для нескольких юзеров, например, для 5,6,7

In [32]:
svd.recommend([5, 6, 7], N=4)

[[2972, 6983, 126088, 124851],
 [91355, 162344, 6611, 120138],
 [4495, 8804, 95311, 5416]]

Если нужно оперировать не id-шниками, а прям названиями:

In [33]:
df_movies = pd.read_csv(DATASET_DIR + '/movies.csv', index_col = "movieId")
df_movies

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy
...,...,...
193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
193585,Flint (2017),Drama


In [74]:
# Найдем фильмы похожие на movieId=2
movie_id = 2
movie_name = df_movies.loc[movie_id]['title']
print("На фильм '{}' по оценкам пользователей похожи следующие фильмы:".format(movie_name))
sims = svd.topN_similar(x=movie_id, N=5, column='item')
for sim in sims:
    print('{} - {} - {}'.format(sim[0], df_movies.loc[sim[0]]['title'], df_movies.loc[sim[0]]['genres']))

На фильм 'Jumanji (1995)' по оценкам пользователей похожи следующие фильмы:
1 - Toy Story (1995) - Adventure|Animation|Children|Comedy|Fantasy
2671 - Notting Hill (1999) - Comedy|Romance
4306 - Shrek (2001) - Adventure|Animation|Children|Comedy|Fantasy|Romance
2424 - You've Got Mail (1998) - Comedy|Romance
2572 - 10 Things I Hate About You (1999) - Comedy|Romance
